In [ ]:
#runtime>change runtime type> gpu
#if you've found this, there is no guarentee anything will work
%tensorflow_version 1.x
!pip install stable_baselines
#need to be tf 1.15
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir /content/pics

     |████████████████████████████████| 412.3MB 37kB/s 
     |████████████████████████████████| 512kB 48.7MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 3.8MB 44.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ad30264ac8d2d63d64aa80f45f4b6624ba9a0a6c703918c88a352e15452ef3d0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [ ]:
!rm -r -f /content/ppo2_robot_tensorboard

In [ ]:
LOG_DIR = './ppo2_robot_tensorboard/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [ ]:
"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R
"""

import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np
import random
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.patches as patches
import matplotlib.pyplot as plt



class CartPoleEnv(gym.Env):

    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5  # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.04  # seconds between state updates
        self.kinematics_integrator = 'euler'
        self.n=0
        self.rF2=0


        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 210.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds.
        high = np.array([self.x_threshold,
                         np.finfo(np.float32).max,
                         self.x_threshold,
                         np.finfo(np.float32).max,
                         self.theta_threshold_radians,
                         np.finfo(np.float32).max,
                         200,200,300,self.theta_threshold_radians,3.1416,200,200],
                        dtype=np.float32)

        self.action_space = spaces.Box(np.array([-1,-1,-1]),np.array([1,1,1]), dtype=np.float32)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)

        self.seed()
        self.viewer = None
        self.state = None
        self.rT=0
        self.subtract=100
        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
    def correctAngle(self,ang):
      while ang>math.pi:
          ang-=math.pi*2
      while ang<=-math.pi:
          ang+=math.pi*2
      return ang
    def newTarget(self,x,y):
        xt=random.random()*400-200
        yt=random.random()*400-200
        while math.hypot(yt-y, xt-x)<100:
          xt=random.random()*400-200
          yt=random.random()*400-200
        return xt,yt
    
    def step(self, action):
        #err_msg = "%r (%s) invalid" % (action, type(action))
        #assert self.action_space.contains(action), err_msg

        x, x_dot, y,y_dot,theta, theta_dot,targetX, targetY,timestep,targetAng, diff,nxt,nyt = self.state
        

        x_dot += np.clip((action[0]-x_dot)*5,-1,1)  * 0.65*self.tau
        y_dot += np.clip((action[1]-y_dot)*5,-1,1) * self.tau
        theta_dot += np.clip((diff*0.8+action[2]-theta_dot)*4,-1,1)*4*self.tau
        x_dot *=(1.0 - (self.tau)) 
        y_dot *=(1.0 - (self.tau)) 
        theta_dot *=1.0 - (self.tau)


        totalSpeed = math.hypot(x_dot, y_dot);
        angle = math.atan2(y_dot, x_dot)-math.radians(90);
        outputAngle = theta + angle;
        x += totalSpeed * math.cos(outputAngle)* self.tau * 200
        y += totalSpeed * math.sin(outputAngle)* self.tau * 200
        theta += theta_dot * self.tau * 1.28471338

        theta=self.correctAngle(theta)
        
        hyp=(math.hypot(targetY-y,targetX-x))

        if False:#self.rF2==0:

              
              reward=-hyp*hyp if hyp>40 else 10**4
              #+0.05*math.hypot(x_dot,y_dot)#np.clip(-hyp*hyp,-1000000,-10*10-10)#-math.fabs(diff/3)-#
              
              #if math.fabs(x)>240 or math.fabs(y)>240:
                #reward=-3
                #reward=
                #timestep-=70
                #done=True
                #targetX,targetY=self.newTarget(x,y)
              done = bool(
                  hyp<40
                  or 
                  timestep>200
                  #or math.fabs(x)>600
                  #or math.fabs(y)>600
              )
              timestep += 1
        else:
              done = bool(
                  timestep>130
                  or math.fabs(x)>300
                  or math.fabs(y)>300
              )
              #   dont kys; dont go too far away
              reward=0#.0002-0.000001*math.floor(hyp*20)/20
              if hyp<40:
                reward=1
                timestep-=self.subtract
                self.subtract*=0.9
                targetX=nxt
                targetY=nyt
                nxt,nyt=self.newTarget(targetX,targetY)

              timestep += 1

        
        targetAng= self.correctAngle(math.atan2(targetY-y, targetX-x)-math.radians(90))
        diff=self.correctAngle(targetAng-theta+math.radians(90))
        
        self.state = (x, x_dot, y,y_dot,theta, theta_dot,targetX, targetY,timestep,targetAng,diff,nxt,nyt)
        self.rT+=1
        return np.array(self.state), reward, done, {}


    def reset(self):
        self.rT=0
        #np.random.seed(0)
        #random.seed(0)
        self.subtract=100
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(6,))
        self.state[0]=random.random()*200-100
        self.state[2]=random.random()*200-100
        self.state = np.append(self.state,self.newTarget(self.state[0],self.state[2]))
        self.state[4]=random.random()*6.28-3.14
        self.state = np.append(self.state,np.zeros(3))
        self.state = np.append(self.state,self.newTarget(self.state[6],self.state[7]))
        self.state[9]=math.atan2(self.state[6]-self.state[0], self.state[7]-self.state[2])
        self.steps_beyond_done = None
        plt.show()
        plt.close()
        return np.array(self.state)

    def render(self, mode='human'):
      self.rF2=1
      x, x_dot, y,y_dot,theta, theta_dot,targetX, targetY,timestep,targetAng,diff,nxt,nyt = self.state

      rect = patches.Rectangle((x-9, y-9),18,10,math.degrees(theta))

      rect2 = patches.Rectangle((targetX-1.5, targetY-1.5),3,3,math.degrees(0), color='red')
      rect3 = patches.Rectangle((nxt-1.5, nyt-1.5),3,3,math.degrees(0), color='purple')

      rect4 = patches.Rectangle((x-9, y-9),1,38,math.degrees(targetAng), color='green')
      rect5 = patches.Rectangle((x-9, y-9),1,38,math.degrees(diff), color='orange')

      # Create figure and axes
      ax = plt.gca()


      # Add the patch to the Axes
      ax.add_patch(rect)
      ax.add_patch(rect2)
      ax.add_patch(rect3)
      ax.add_patch(rect4)
      ax.add_patch(rect5)
      plt.xlim([-300, 300])
      plt.ylim([-300, 300])

      plt.savefig("/content/pics/file%05d.png" % self.n)
      plt.show()
      self.n+=1
      plt.close()
    def close(self):
      self.n=0

  



In [ ]:
%cd /content/
import gym
import json
import datetime as dt
import glob
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
#from stable_baselines.common.vec_env import SubprocVecEnv
import copy 


from stable_baselines import PPO2
from stable_baselines.common.schedules import PiecewiseSchedule
from stable_baselines.common.schedules import LinearSchedule

import pandas as pd
#             learning_rate = sched_LR.value,

sched_LR = LinearSchedule(300000,0.00005,0.005)#
sched_LR =PiecewiseSchedule([[0,0.000009],[0.15, 0.00005],[ 0.6, 0.0003],[1, 0.00003]],outside_value=0.0005)

#for i in np.linspace(0.1,0.4,15):
env=DummyVecEnv([lambda: CartPoleEnv()])
model = PPO2(MlpPolicy,
             env,
             ent_coef=0.03,
             #learning_rate = 0.0005,#sched_LR.value,
             verbose=0, 
             tensorboard_log="./ppo2_robot_tensorboard/")
for i in np.linspace(0,1,6): 
  model.learn(total_timesteps=750)

model.save("ppo2_stupidBot")

/content
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







In [ ]:
!rm /content/pics/*

obs = env.reset()

for i in range(2000):
  action, _states = model.predict(obs)
  obs, rewards, done, info = env.step(action)
  #obs[0][8]=0
  env.render()
env.close()
%cd /content/pics


!ffmpeg -y -framerate 60 -i file%05d.png -r 30 -pix_fmt yuv420p video_name.mp4 -loglevel fatal

%cd /content/


from IPython.display import HTML
from base64 import b64encode
import os
# Compressed video path
save_path= "/content/pics/video_name.mp4"
compressed_path = "/content/pics/video_name.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

rm: cannot remove '/content/pics/*': No such file or directory
/content/pics
/content


In [ ]:
while True:pass

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
for key, value in model.get_parameters().items():
    print (key, np.shape(value))


In [ ]:
import random
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib as mpl

import math
from datetime import datetime
#stolen from RR lite just a basic spline generation thing
def spline_fit(x0, dx0, x1, dx1):
    a = 2 * x0 + dx0 - 2 * x1 + dx1
    b = -3 * x0 - 2 * dx0 + 3 * x1 - dx1
    c = dx0
    d = x0
    return a, b, c, d

def spline_get(spline, u):
    a, b, c, d = spline
    return a * u**3 + b * u**2 + c * u + d

def spline_deriv(spline, u):
    a, b, c, d = spline
    return 3 * a * u**2 + 2 * b * u + c


class RobotEnv(gym.Env):
  """A robot trajectory following enviornment based on RoadRunner for OpenAI gym"""
  metadata = {'render.modes': ['human']}

  def getNewSpline(self):
    self.x1=self.ROBOT_X
    self.y1=self.ROBOT_Y
    self.angle1=self.ROBOT_O
    #while math.hypot(self.y2-self.ROBOT_Y,self.x2-self.ROBOT_X)<50:
    self.x2=random.random()*500-250
    self.y2=random.random()*500-250
    self.angle2=self.angle1#random.random()*6.28

    x_spline = spline_fit(self.x1, math.cos(self.angle1)*100, self.x2, math.cos(self.angle2)*100)
    y_spline = spline_fit(self.y1, math.sin(self.angle1)*100, self.y2, math.sin(self.angle2)*100)
    u = np.linspace(0, 1, 100)

    self.x_splinePts=spline_get(x_spline, u)
    self.y_splinePts=spline_get(y_spline, u)
    self.x_splinePts=np.append(self.x_splinePts,np.full((1, 10), self.x2))
    self.y_splinePts=np.append(self.y_splinePts,np.full((1, 10), self.y2))

    self.ghyp=math.hypot(self.x1-self.x2,self.y1-self.y2)



  def __init__(self):
    super(RobotEnv, self).__init__()    
    self.current_step = 1

    self.thisTime=0
    self.ROBOT_X=random.random()*500-250
    self.ROBOT_Y=random.random()*500-250
    self.ROBOT_DX=0
    self.ROBOT_DY=0
    self.ROBOT_O=0
    self.ROBOT_DO=0

    self.x1=self.ROBOT_X
    self.y1=self.ROBOT_Y
    self.angle1=self.ROBOT_O
    random.seed(datetime.now())


    self.x2=random.random()*500-250
    self.y2=random.random()*500-250
    self.angle2=self.angle1#random.random()*6.28

    x_spline = spline_fit(self.x1, math.cos(self.angle1)*100, self.x2, math.cos(self.angle2)*100)
    y_spline = spline_fit(self.y1, math.sin(self.angle1)*100, self.y2, math.sin(self.angle2)*100)
    u = np.linspace(0, 1, 100)

    self.x_splinePts=spline_get(x_spline, u)
    self.y_splinePts=spline_get(y_spline, u)

    self.getNewSpline()
    self.targetReached=False
    self.numReached=0
    


    self.reward_range = (0, 10)

    self.action_space = spaces.Box(np.array([-1.0,-1.0,-1.0]), np.array([1.0,1.0,1.0]), dtype=np.float32)
    high=np.full((1, 8), 250, np.float32)
    self.observation_space = spaces.Box(np.array([-250.0,-250.0,-math.pi,-250.0,-250.0,-10.0,-250.0,-250.0,-math.pi]),np.array([250,250,math.pi,250,250,10,250,250,math.pi]), dtype=np.float32)




  def _next_observation(self):
    # info about where the robot is
    state = np.array( [self.ROBOT_X,
                      self.ROBOT_Y,
                      self.ROBOT_O,
                      self.ROBOT_DX,
                      self.ROBOT_DY,
                      self.ROBOT_DO]
    )


    # Append data about the trajectory lookahead
    #obs=np.append(state,np.array([self.x2-self.ROBOT_X,self.y2-self.ROBOT_Y]))
    ##stepp=self.current_step%100
    ##obs = np.append(state, self.x_splinePts[stepp:stepp+5])
    ##obs = np.append(obs, self.y_splinePts[stepp:stepp+5])
    angD=self.ROBOT_O-math.atan2(self.y2-self.ROBOT_Y,self.x2-self.ROBOT_X)
    while angD>=math.pi:
      angD-=math.pi*2
    while angD<-math.pi:
      angD+=math.pi*2
    #self.ROBOT_DO-=angD
    
    return np.array(np.append(state,np.array([self.x2,self.y2,angD])))#[self.x2-self.ROBOT_X,self.y2-self.ROBOT_Y,angD,math.sin(self.ROBOT_O),math.cos(self.ROBOT_O)])

  def _take_action(self, action):
    # Set the current price to a random price within the time step

    totalSpeed = math.hypot(self.ROBOT_DX, self.ROBOT_DY);
    angle = math.atan2(self.ROBOT_DY, self.ROBOT_DX);
    outputAngle = self.ROBOT_O + angle;
    self.ROBOT_X += totalSpeed * math.cos(outputAngle);
    self.ROBOT_Y += totalSpeed * math.sin(outputAngle);
    self.ROBOT_O += self.ROBOT_DO/62;
    self.ROBOT_DX += action[0]* 0.9;
    self.ROBOT_DY += action[1]* 0.65;
    self.ROBOT_DO += (action[2]* 0.5*0);
    self.ROBOT_DX *=0.96;
    self.ROBOT_DY *=0.96;
    self.ROBOT_DO *=0.96;
    while self.ROBOT_O>math.pi:
      self.ROBOT_O-=math.pi*2
    while self.ROBOT_O<-math.pi:
      self.ROBOT_O+=math.pi*2

    #gf sim code


  def getReward(self,hyp):
    #we want robot to complete path in 100 timesteps
    #xSplinePt=self.x_splinePts[int((self.current_step%100))]
    #ySplinePt=self.y_splinePts[int((self.current_step%100))]

    #if hyp<1:
    #  self.targetReached=True
    #  self.numReached+=1
    #  self.getNewSpline()
      



    #hyp=hyp*((self.current_step%100)+1)/50

    #return 1/math.fabs(hyp*0.0047+1/10)-self.ROBOT_DO
    #maybe discretize is good?https://www.alexirpan.com/2018/02/14/rl-hard.html
    return (1 if hyp<1 else 0)#1/((hyp-10)*(hyp-10)*0.01+1/10)+self.numReached*10#+self.numReached###+(0.03 if math.fabs(angD)<0.5 else 0) #  #math.max(10,1/hyp)

  def step(self, action):
    # Execute one time step within the environment

    self._take_action(action)

    self.current_step += 1
    self.y2=self.y_splinePts[int((self.current_step%100))]
    self.x2=self.x_splinePts[int((self.current_step%100))]


    hyp=math.hypot(self.y2-self.ROBOT_Y,self.x2-self.ROBOT_X)
    reward = 1 if self.ROBOT_X-self.y1>100 else 0#self.getReward(hyp)
    done=False
    if not self.targetReached:
      done = bool((self.current_step%100)==99)
    #else:
    #  self.targetReached=False
    #if self.ghyp+3-(self.current_step%100)/100*self.ghyp<hyp:
       #done=True
    if done:
      self.getNewSpline()
      self.numReached=0
      #self.current_step+=100-(self.current_step%100)


    obs = self._next_observation()

    
    return obs, reward, done, {}

  def reset(self):
    # Reset the state of the environment to an initial state
    self.ROBOT_X=random.random()*500-250
    self.ROBOT_Y=random.random()*500-250
    self.ROBOT_DX=0
    self.ROBOT_DY=0
    self.ROBOT_O=0
    self.ROBOT_DO=0

    self.x1=random.random()*500-250
    self.x2=random.random()*500-250
    self.y1=random.random()*500-250
    self.y2=random.random()*500-250
    self.getNewSpline()

    # Set the current step to a random point within the data frame

    return self._next_observation()

  def render(self, mode='human', close=False):
    # Render the environment to the screen
    rect = patches.Rectangle((self.ROBOT_X, self.ROBOT_Y),18,18,math.degrees(self.ROBOT_O))

    plt.title('Robot')
    plt.xlabel('x [in]')
    plt.ylabel('y [in]')
    # Create figure and axes
    ax = plt.gca()


    # Add the patch to the Axes
    ax.add_patch(rect)

    plt.plot(self.x_splinePts, self.y_splinePts)
    plt.xlim([-250, 250])
    plt.ylim([-250, 250])
    plt.show()
    plt.close()






  #    public void update() {
  #   long currentTimeMillis = System.currentTimeMillis();
  #   double elapsedTime = (double)(currentTimeMillis - this.lastUpdateTime) / 1000.0D;
  #   this.lastUpdateTime = currentTimeMillis;
  #   if (!(elapsedTime > 1.0D)) {
  #      double totalSpeed = Math.hypot(xSpeed, ySpeed);
  #      double angle = Math.atan2(ySpeed, xSpeed) - Math.toRadians(90.0D);
  #      double outputAngle = worldAngle_rad + angle;
  #      worldXPosition += totalSpeed * Math.cos(outputAngle) * elapsedTime * 1000.0D * 0.2D;
  #      worldYPosition += totalSpeed * Math.sin(outputAngle) * elapsedTime * 1000.0D * 0.2D;
  #      worldAngle_rad += MovementVars.movement_turn * elapsedTime * 20.0D / 6.283185307179586D;
  #      xSpeed += Range.clip((MovementVars.movement_x - xSpeed) / 0.2D, -1.0D, 1.0D) * elapsedTime;
  #      ySpeed += Range.clip((MovementVars.movement_y - ySpeed) / 0.2D, -1.0D, 1.0D) * elapsedTime;
  #      turnSpeed += Range.clip((MovementVars.movement_turn - turnSpeed) / 0.2D, -1.0D, 1.0D) * elapsedTime;
  #      xSpeed *= 1.0D - elapsedTime;
  #      ySpeed *= 1.0D - elapsedTime;
  #      turnSpeed *= 1.0D - elapsedTime;
  #   }
  #}











#         done = bool(
#           self.rT>1000
#           or math.fabs(x)>350
#           or math.fabs(y)>350
#       )
#       hyp=(math.hypot(targetY-y,targetX-x))
#       reward=0
#       if self.f>0:
#         reward = 1/(hyp*hyp*0.001+1)
#       elif self.f==0:
#         targetX,targetY=self.newTarget(x,y)
#       elif hyp<40:
#         reward=0.5+(100-timestep)/100
#         timestep=0
#         self.f=3
#       else:
#         timestep += 1
#       self.f-=1
#       targetAng= math.atan2(targetY-y, targetX-x)
#
#       self.state = (x, x_dot, y,y_dot,theta, theta_dot,targetX, targetY,timestep,targetAng)
#       self.rT+=1
#       return np.array(self.state), reward, done, {}
env=DummyVecEnv([lambda: CartPoleEnv()])
modelR = PPO2(MlpPolicy, env,ent_coef=i,learning_rate = 0.0005,verbose=0, tensorboard_log="./a2c_robot_tensorboard/")
for j in np.linspace(0.01, 0.04, num=10):
  bestR=0
  for i in np.linspace(0.01, 0.04, num=4):
    env=DummyVecEnv([lambda: CartPoleEnv()])	
    model = copy.copy(modelR)
	model.ent_coef=i
    model.learn(total_timesteps=20000)
    r=sum(model.episode_reward)/len(model.episode_reward)
    if r>bestR:
      bestR=r
      modelR=copy.copy(model)
      

In [ ]:

        done = bool(
            self.rT>1000
            or math.fabs(x)>250
            or math.fabs(y)>250
        )
        hyp=(math.hypot(targetY-y,targetX-x))
        reward=0
        if hyp<20:
          t=(100-timestep)
          reward=1+5184/((t+36)*(t+36))
          timestep=0
          targetX,targetY=self.newTarget(x,y)
        else:
          timestep += 1
        targetAng= math.atan2(targetY-y, targetX-x)

        self.state = (x, x_dot, y,y_dot,theta, theta_dot,targetX, targetY,timestep,targetAng)
        self.rT+=1
        return np.array(self.state), reward, done, {}